In [1]:
import h5py
import torch
import shutil
import os 

In [2]:
def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    model_name = task_id + "model_best.pth.tar"
    # model_name = task_id + "model_best_hmn.pth.tar"
    # base_path = 'D:/Dev/project/roofcounting/'
    base_path = '/home/jthe/Dev/project/roofcounting/'
#     base_path = ('/content/drive/MyDrive/CSRNet-pytorch/')  
    torch.save(state, task_id+filename)
    if is_best:
        shutil.copyfile(task_id+filename, os.path.join(base_path,model_name))            

In [3]:
from torch import nn
import torch
from torchvision import models


class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
#            for i in xrange(len(self.frontend.state_dict().items())):
            for i in range(len(self.frontend.state_dict().items())):
#                self.frontend.state_dict().items()[i][1].data[:] = mod.state_dict().items()[i][1].data[:]
                list(self.frontend.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i][1].data[:] 
    
    def forward(self,x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)                


In [4]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import cv2

def load_data(img_path,train = True):
    gt_path = img_path.replace('.png','.h5').replace('images','ground_truth')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path)
    target = np.asarray(gt_file['density'])
    if False:
        crop_size = (img.size[0]/2,img.size[1]/2)
        if random.randint(0,9)<= -1:
            
            
            dx = int(random.randint(0,1)*img.size[0]*1./2)
            dy = int(random.randint(0,1)*img.size[1]*1./2)
        else:
            dx = int(random.random()*img.size[0]*1./2)
            dy = int(random.random()*img.size[1]*1./2)
        
        
        
        img = img.crop((dx,dy,crop_size[0]+dx,crop_size[1]+dy))
        target = target[dy:crop_size[1]+dy,dx:crop_size[0]+dx]
        
        
        
        
        if random.random()>0.8:
            target = np.fliplr(target)
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    
    
    target = cv2.resize(target,(target.shape[1]//8,target.shape[0]//8),interpolation = cv2.INTER_CUBIC)*64
    
    
    return img,target


In [5]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
# from image import *
import torchvision.transforms.functional as F

class listDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, seen=0, batch_size=1, num_workers=4):
        if train:
            root = root *4
        random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.seen = seen
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        
    def __len__(self):
        return self.nSamples
    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
           
        
        if self.transform is not None:
            img = self.transform(img)
        return img,target

In [6]:
import sys
import os
import warnings

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

import numpy as np
import json
import cv2
import time

import gc

gpu = "0"
# gpu = ""
best_prec1 = 1e6
original_lr = 1e-7
lr = 1e-7
batch_size = 1
momentum = 0.95
decay = 5*1e-4
start_epoch = 0
epochs = 400
steps = [-1, 1, 100, 150]
scales = [1, 1, 1, 1]
workers = 4
seed = time.time()
print_freq = 30
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
torch.cuda.manual_seed(seed)

model = CSRNet()

if gpu != "":
  model = model.cuda()
  criterion = nn.MSELoss(size_average=False).cuda()
  
else:
  model = model.cpu()
  criterion = nn.MSELoss(size_average=False).cpu()

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)


/home/jthe/anaconda3/envs/roofcount/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jthe/anaconda3/envs/roofcount/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jthe/anaconda3/envs/roofcount/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
def train(train_list, model, criterion, optimizer, epoch):
    
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    
    
    train_loader = torch.utils.data.DataLoader(
        listDataset(train_list,
                       shuffle=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]), 
                       train=True, 
                       seen=model.seen,
                       batch_size=batch_size,
                       num_workers=workers),
        batch_size=batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), lr))
    
    model.train()
    end = time.time()

    if gpu != "":
      gc.collect()

    for i,(img, target)in enumerate(train_loader):
        if gpu != "":
          torch.cuda.empty_cache()
        # print("i: {}".format(i))
        data_time.update(time.time() - end)
        
        if gpu != "":
          img = img.cuda()
        else:
          img = img.cpu()

        img = Variable(img)
        output = model(img)
        
        
        
        if gpu != "":
          target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        else:
          target = target.type(torch.FloatTensor).unsqueeze(0).cpu()
          
        target = Variable(target)
        
        
        loss = criterion(output, target)
        
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    
def validate(val_list, model, criterion):
    print ('begin test')
    print("clearing gpu cache")
    
    if gpu != "":
      torch.cuda.empty_cache() 

    test_loader = torch.utils.data.DataLoader(
    listDataset(val_list,
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]),  train=False),
    batch_size=batch_size)    
    
    model.eval()
    
    mae = 0
    
    for i,(img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)
        
        mae += abs(output.data.sum()-target.sum().type(torch.FloatTensor).cuda())
        
    mae = mae/len(test_loader)    
    print(' * MAE {mae:.3f} '
              .format(mae=mae))

    return mae    
        
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    
    
    lr = original_lr
    
    for i in range(len(steps)):
        
        scale = scales[i] if i < len(scales) else 1
        
        
        if epoch >= steps[i]:
            lr = lr * scale
            if epoch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count    

In [9]:
# base_path = 'D:/Dev/project/roofcounting/'
base_path = '/home/jthe/Dev/project/roofcounting/'
train_json_path = os.path.join(base_path, "notebook/dev", "part_A_train.json")
val_json_path = os.path.join(base_path, "notebook/dev", "part_A_with_val.json")

with open(train_json_path, 'r') as outfile:
  train_list = json.load(outfile)

with open(val_json_path, 'r') as outfile:
  val_list = json.load(outfile)


In [10]:
task = "0"

pre = 0 # swap this with 0 to train from scratch, 1 for using pre-trained model
pre_trained_path = "0model_best.pth.tar"

pre_file = os.path.join(base_path, pre_trained_path)
if pre:
  if os.path.isfile(pre_file):
    print("=> loading checkpoint '{}'".format(pre_file))

    if gpu != "":
      checkpoint = torch.load(pre_file)
    else:
      checkpoint = torch.load(pre_file, map_location=torch.device("cpu"))
      
    
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}'".format(pre_file, checkpoint['epoch']))

  else:
    print("=> no checkpoint found at '{}'".format(pre_file))


for epoch in range(start_epoch, epochs):
  adjust_learning_rate(optimizer, epoch)

  train(train_list, model, criterion, optimizer, epoch)
  prec1 = validate(val_list, model, criterion)

  is_best = prec1 < best_prec1
  best_prec1 = min(prec1, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
      'epoch': epoch + 1, 
      'arch': None, 
      'state_dict' : model.state_dict(),
      'best_prec1' : best_prec1,
      'optimizer' : optimizer.state_dict(),
  }, is_best, task)

epoch 0, processed 0 samples, lr 0.0000001000
Epoch: [0][0/392]	Time 0.534 (0.534)	Data 0.095 (0.095)	Loss 44.7435 (44.7435)	
Epoch: [0][30/392]	Time 0.254 (0.259)	Data 0.167 (0.157)	Loss 11.2531 (42.7101)	
Epoch: [0][60/392]	Time 0.255 (0.257)	Data 0.168 (0.162)	Loss 8.4442 (60.3359)	
Epoch: [0][90/392]	Time 0.255 (0.256)	Data 0.168 (0.164)	Loss 18.5251 (62.9563)	
Epoch: [0][120/392]	Time 0.255 (0.256)	Data 0.168 (0.165)	Loss 32.4760 (59.6058)	
Epoch: [0][150/392]	Time 0.255 (0.255)	Data 0.168 (0.165)	Loss 138.7124 (62.0622)	
Epoch: [0][180/392]	Time 0.255 (0.255)	Data 0.168 (0.166)	Loss 6.3330 (65.2548)	
Epoch: [0][210/392]	Time 0.255 (0.255)	Data 0.168 (0.166)	Loss 65.1478 (61.4180)	
Epoch: [0][240/392]	Time 0.255 (0.255)	Data 0.168 (0.166)	Loss 22.7605 (61.5961)	
Epoch: [0][270/392]	Time 0.255 (0.255)	Data 0.168 (0.166)	Loss 101.8116 (60.6249)	
Epoch: [0][300/392]	Time 0.255 (0.255)	Data 0.168 (0.167)	Loss 15.9598 (64.4049)	
Epoch: [0][330/392]	Time 0.255 (0.255)	Data 0.168 (0.167)

Epoch: [6][240/392]	Time 0.256 (0.255)	Data 0.168 (0.168)	Loss 26.9110 (66.9235)	
Epoch: [6][270/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 15.0907 (63.3827)	
Epoch: [6][300/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 26.2297 (62.6396)	
Epoch: [6][330/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 30.6031 (62.7152)	
Epoch: [6][360/392]	Time 0.256 (0.255)	Data 0.168 (0.168)	Loss 11.3747 (63.0770)	
Epoch: [6][390/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 5.7169 (61.9568)	
begin test
clearing gpu cache
 * MAE 152.228 
 * best MAE 152.228 
epoch 7, processed 2744 samples, lr 0.0000001000
Epoch: [7][0/392]	Time 0.183 (0.183)	Data 0.096 (0.096)	Loss 25.2429 (25.2429)	
Epoch: [7][30/392]	Time 0.256 (0.253)	Data 0.169 (0.166)	Loss 3.2741 (48.5784)	
Epoch: [7][60/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 283.5776 (60.7418)	
Epoch: [7][90/392]	Time 0.256 (0.254)	Data 0.169 (0.167)	Loss 706.1688 (66.1327)	
Epoch: [7][120/392]	Time 0.256 (0.254)	Data 0.169 (0.168)	Loss 51.071

Epoch: [13][0/392]	Time 0.184 (0.184)	Data 0.097 (0.097)	Loss 54.7894 (54.7894)	
Epoch: [13][30/392]	Time 0.255 (0.253)	Data 0.169 (0.166)	Loss 3.7071 (22.5143)	
Epoch: [13][60/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 22.0956 (30.6393)	
Epoch: [13][90/392]	Time 0.256 (0.254)	Data 0.169 (0.167)	Loss 46.3204 (38.6011)	
Epoch: [13][120/392]	Time 0.256 (0.254)	Data 0.169 (0.168)	Loss 3.1575 (42.7538)	
Epoch: [13][150/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 500.5976 (47.0808)	
Epoch: [13][180/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 49.0322 (46.5208)	
Epoch: [13][210/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 81.8199 (48.9893)	
Epoch: [13][240/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 18.6013 (46.3482)	
Epoch: [13][270/392]	Time 0.255 (0.255)	Data 0.169 (0.168)	Loss 49.1911 (44.3420)	
Epoch: [13][300/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 57.5709 (43.7414)	
Epoch: [13][330/392]	Time 0.255 (0.255)	Data 0.169 (0.168)	Loss 20.8491 (43.7827)	
Epoch: [13

Epoch: [19][240/392]	Time 0.255 (0.255)	Data 0.169 (0.168)	Loss 82.6900 (44.3111)	
Epoch: [19][270/392]	Time 0.255 (0.255)	Data 0.169 (0.168)	Loss 10.1822 (42.8442)	
Epoch: [19][300/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 155.3184 (43.0785)	
Epoch: [19][330/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 10.3087 (44.5144)	
Epoch: [19][360/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 4.8260 (44.9612)	
Epoch: [19][390/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 93.9824 (44.7461)	
begin test
clearing gpu cache
 * MAE 72.328 
 * best MAE 72.328 
epoch 20, processed 7840 samples, lr 0.0000001000
Epoch: [20][0/392]	Time 0.184 (0.184)	Data 0.097 (0.097)	Loss 3.2291 (3.2291)	
Epoch: [20][30/392]	Time 0.255 (0.253)	Data 0.169 (0.166)	Loss 48.1913 (19.8122)	
Epoch: [20][60/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 383.3372 (39.9517)	
Epoch: [20][90/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 0.2640 (39.4212)	
Epoch: [20][120/392]	Time 0.256 (0.254)	Data 0.169 (0.168)	Los

Epoch: [26][0/392]	Time 0.175 (0.175)	Data 0.087 (0.087)	Loss 151.7441 (151.7441)	
Epoch: [26][30/392]	Time 0.255 (0.252)	Data 0.169 (0.166)	Loss 0.8482 (71.4112)	
Epoch: [26][60/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 6.2747 (55.0270)	
Epoch: [26][90/392]	Time 0.256 (0.254)	Data 0.169 (0.167)	Loss 10.3465 (53.5472)	
Epoch: [26][120/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 5.2161 (46.1802)	
Epoch: [26][150/392]	Time 0.256 (0.254)	Data 0.169 (0.168)	Loss 3.3076 (44.2348)	
Epoch: [26][180/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 3.2926 (44.1269)	
Epoch: [26][210/392]	Time 0.255 (0.255)	Data 0.169 (0.168)	Loss 104.0220 (42.9165)	
Epoch: [26][240/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 9.3476 (42.5866)	
Epoch: [26][270/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 370.2690 (43.0822)	
Epoch: [26][300/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 2.7465 (43.2392)	
Epoch: [26][330/392]	Time 0.256 (0.255)	Data 0.169 (0.168)	Loss 41.2245 (44.3794)	
Epoch: [26][

...

Epoch: [369][330/392]	Time 0.255 (0.254)	Data 0.169 (0.168)	Loss 0.8056 (4.1641)	
Epoch: [369][360/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 3.4191 (4.1747)	
Epoch: [369][390/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 11.2465 (4.2582)	
begin test
clearing gpu cache
 * MAE 46.306 
 * best MAE 36.522 
epoch 370, processed 145040 samples, lr 0.0000001000
Epoch: [370][0/392]	Time 0.177 (0.177)	Data 0.090 (0.090)	Loss 1.3625 (1.3625)	
Epoch: [370][30/392]	Time 0.255 (0.252)	Data 0.168 (0.165)	Loss 0.8294 (3.8401)	
Epoch: [370][60/392]	Time 0.255 (0.253)	Data 0.169 (0.167)	Loss 1.8216 (4.1764)	
Epoch: [370][90/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 11.1268 (4.6904)	
Epoch: [370][120/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 2.3213 (4.5145)	
Epoch: [370][150/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 2.0454 (4.2721)	
Epoch: [370][180/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 0.1064 (4.2645)	
Epoch: [370][210/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 

Epoch: [376][120/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 1.2209 (3.8427)	
Epoch: [376][150/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 1.2691 (3.8173)	
Epoch: [376][180/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 7.7051 (3.7415)	
Epoch: [376][210/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 0.8129 (3.7680)	
Epoch: [376][240/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 10.5969 (3.7939)	
Epoch: [376][270/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 8.7008 (3.7348)	
Epoch: [376][300/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 4.2001 (3.6478)	
Epoch: [376][330/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 2.4243 (3.6711)	
Epoch: [376][360/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 0.0613 (3.7409)	
Epoch: [376][390/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 2.7643 (3.7311)	
begin test
clearing gpu cache
 * MAE 38.807 
 * best MAE 36.522 
epoch 377, processed 147784 samples, lr 0.0000001000
Epoch: [377][0/392]	Time 0.183 (0.183)	Data 0.096 (0.096)	Los

Epoch: [382][360/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 1.3830 (3.4684)	
Epoch: [382][390/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 7.4340 (3.4764)	
begin test
clearing gpu cache
 * MAE 39.585 
 * best MAE 36.522 
epoch 383, processed 150136 samples, lr 0.0000001000
Epoch: [383][0/392]	Time 0.185 (0.185)	Data 0.098 (0.098)	Loss 2.2583 (2.2583)	
Epoch: [383][30/392]	Time 0.255 (0.252)	Data 0.169 (0.166)	Loss 0.8528 (3.1288)	
Epoch: [383][60/392]	Time 0.255 (0.253)	Data 0.168 (0.167)	Loss 0.0795 (3.1351)	
Epoch: [383][90/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 7.5086 (3.3206)	
Epoch: [383][120/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 0.6509 (3.4030)	
Epoch: [383][150/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 4.7663 (3.2869)	
Epoch: [383][180/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 8.0205 (3.4488)	
Epoch: [383][210/392]	Time 0.255 (0.254)	Data 0.169 (0.168)	Loss 3.0737 (3.4664)	
Epoch: [383][240/392]	Time 0.255 (0.254)	Data 0.169 (0.168)	Loss 2.

Epoch: [389][150/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 3.1747 (4.0433)	
Epoch: [389][180/392]	Time 0.257 (0.254)	Data 0.168 (0.167)	Loss 1.1912 (3.9652)	
Epoch: [389][210/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 0.6234 (3.9591)	
Epoch: [389][240/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 6.2043 (4.0034)	
Epoch: [389][270/392]	Time 0.255 (0.254)	Data 0.169 (0.168)	Loss 2.8626 (4.0869)	
Epoch: [389][300/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 1.1605 (4.2932)	
Epoch: [389][330/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 9.6353 (4.5318)	
Epoch: [389][360/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 8.6057 (4.5425)	
Epoch: [389][390/392]	Time 0.255 (0.254)	Data 0.169 (0.168)	Loss 8.9498 (4.4946)	
begin test
clearing gpu cache
 * MAE 43.526 
 * best MAE 36.522 
epoch 390, processed 152880 samples, lr 0.0000001000
Epoch: [390][0/392]	Time 0.182 (0.182)	Data 0.095 (0.095)	Loss 9.5201 (9.5201)	
Epoch: [390][30/392]	Time 0.255 (0.252)	Data 0.168 (0.166)	Loss 

Epoch: [395][390/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 8.3266 (3.6694)	
begin test
clearing gpu cache
 * MAE 40.392 
 * best MAE 36.522 
epoch 396, processed 155232 samples, lr 0.0000001000
Epoch: [396][0/392]	Time 0.175 (0.175)	Data 0.088 (0.088)	Loss 3.5697 (3.5697)	
Epoch: [396][30/392]	Time 0.255 (0.252)	Data 0.168 (0.165)	Loss 2.1637 (3.5328)	
Epoch: [396][60/392]	Time 0.255 (0.253)	Data 0.169 (0.167)	Loss 5.7961 (3.4511)	
Epoch: [396][90/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 2.2657 (3.3903)	
Epoch: [396][120/392]	Time 0.255 (0.254)	Data 0.169 (0.167)	Loss 0.8809 (3.3939)	
Epoch: [396][150/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 2.8727 (3.5523)	
Epoch: [396][180/392]	Time 0.255 (0.254)	Data 0.168 (0.167)	Loss 0.0482 (3.4346)	
Epoch: [396][210/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 5.3137 (3.4305)	
Epoch: [396][240/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 3.1452 (3.4141)	
Epoch: [396][270/392]	Time 0.255 (0.254)	Data 0.168 (0.168)	Loss 0.